# 2.1.1 Zeichenerklärung – OCR (GPT4o)

* Zero-Shot
* Chain-Of-Thought

In [1]:
from utils.parser import Parser
from utils.openai import OpenAI
from utils.pprint import pprint
import asyncio

instructions = "Du bist ein Assistent zur getreuen Wiedergabe von Informationen aus einem Bebauungsplan. Achte auf Vollständigkeit."
jarvis = OpenAI(instructions)
parser = Parser()

In [5]:
# Zeichenerklärung – Schema Nutzungsschablone WITHOUT BOUNDING BOX
ze_tt_path = "../data/processed/bpläne/1_alles_in_einem_dokument/L04-01-ZE-TT-crop.png"
ze_path = "../data/processed/bpläne/1_alles_in_einem_dokument/L04-01-ZE-crop.png"
ze_two_col_path = "../data/processed/bpläne/1_alles_in_einem_dokument/S03-01-ZE-two-col-crop.png"

instruction = 'Gehe Schritt für Schritt vor: 1. Finde den Eintrag "Nutzungsschablone" in der vorliegenden Zeichenerklärung. 2. Finde die zugehörige Tabellenerklärung zum Eintrag "Nutzungsschablone". 3. Gebe die Tabellenerklärung in ASCII-Tabellenform aus.'

messages = await jarvis.extractTextFromImages(instruction, [ze_tt_path, ze_path, ze_two_col_path])
pprint(messages)

1. **Eintrag "Nutzungsschablone" in der Zeichenerklärung:**

   In der Zeichenerklärung auf der linken Seite des Bebauungsplans ist der Eintrag "Nutzungsschablone" zu finden. Er ist durch ein Symbol dargestellt, das wie ein Rechteck mit einer diagonalen Linie aussieht.

2. **Zugehörige Tabellenerklärung zum Eintrag "Nutzungsschablone":**

   Die zugehörige Tabellenerklärung befindet sich auf der rechten Seite des Bebauungsplans unter dem Abschnitt "Planungsrechtliche Festsetzungen". Der Eintrag "Nutzungsschablone" ist unter Punkt 1.4.1 zu finden.

3. **Tabellenerklärung in ASCII-Tabellenform:**

   ```
   +----------------------+----------------------+
   | Nutzungsschablone    |                      |
   +----------------------+----------------------+
   | 1.4.1                |                      |
   +----------------------+----------------------+
   | Nutzungsschablone    | Ist die für Nutzungsschablone eingetragene |
   |                      | Nutzung.                          

In [6]:
# Zeichenerklärung – Schema Nutzungsschablone WITH BOUNDING BOX
ze_tt_bb_path = "../data/processed/bpläne/1_alles_in_einem_dokument/L04-01-ZE-TT-bb-crop.png"
ze_bb_path = "../data/processed/bpläne/1_alles_in_einem_dokument/L04-01-ZE-bb-crop.png"
ze_two_col_bb_path = "../data/processed/bpläne/1_alles_in_einem_dokument/S03-01-ZE-two-col-bb-crop.png"

instruction = 'Gebe die rot umrandete Tabellenerklärung zum Eintrag Nutzungsschablone in ASCII-Tabellenform aus.'

messages = await jarvis.extractTextFromImages(instruction, [ze_tt_bb_path, ze_bb_path, ze_two_col_bb_path])
pprint(messages)

# Store Schema
msg211_l04_zettbb_nz = messages[0]
msg211_l04_zebb_nz = messages[1]
msg211_s03_zetwocolbb_nz = messages[2]
%store msg211_l04_zettbb_nz msg211_l04_zebb_nz msg211_s03_zetwocolbb_nz

Hier ist die rot umrandete Tabellenerklärung zum Eintrag Nutzungsschablone in ASCII-Tabellenform:

```
+-------------------+----------------------+
| Baugebiet         | Zahl der Vollgeschosse|
+-------------------+----------------------+
| Grundflächenzahl  | Geschossflächenzahl  |
+-------------------+----------------------+
| Bauweise          |                      |
+-------------------+----------------------+
| Dachform und      |                      |
| Dachneigung       |                      |
+-------------------+----------------------+
```
#############################################
Hier ist die rot umrandete Tabellenerklärung in ASCII-Tabellenform:

```
+----------------+---------------------+
| Baugebiet      | Zahl der Vollgeschosse |
+----------------+---------------------+
| Grundflächenzahl | Geschossflächenzahl  |
+----------------+---------------------+
| -              | Bauweise             |
+----------------+---------------------+
| Dachform und Dachneigung   

In [7]:
# Zeichenerklärung – Komplette Definition Nutzungsschablone – WITH CONTEXT
# Kontext: Schema
images = [ze_tt_path, ze_path, ze_two_col_path]
contexts = parser.text2prompts([msg211_l04_zettbb_nz, msg211_l04_zebb_nz, msg211_s03_zetwocolbb_nz])

instruction = 'Fasse alle relevanten Definitionen und Zeichen bezüglich der Parameter der ASCII-Nutzungsschablone aus der Zeichenerklärung zusammen. Ausgabe im JSON-Format: {<Parameter>: {"Definition": "", "Zeichen"=""}}'

messages = await jarvis.extractTextFromImagesWithContexts(instruction, images, contexts)
pprint(messages)

Hier sind die relevanten Definitionen und Zeichen bezüglich der Parameter der ASCII-Nutzungsschablone aus der Zeichenerklärung im JSON-Format:

```json
{
  "Baugebiet": {
    "Definition": "Art der baulichen Nutzung",
    "Zeichen": {
      "WR": "Reines Wohngebiet",
      "WA": "Allgemeines Wohngebiet",
      "MI": "Mischgebiet",
      "GE2": "Gewerbegebiet"
    }
  },
  "Zahl der Vollgeschosse": {
    "Definition": "Maximale Anzahl der Vollgeschosse",
    "Zeichen": "römische Ziffern (z.B. II für 2 Vollgeschosse)"
  },
  "Grundflächenzahl": {
    "Definition": "Verhältnis der Grundfläche des Gebäudes zur Grundstücksfläche",
    "Zeichen": "Zahl (z.B. 0,4)"
  },
  "Geschossflächenzahl": {
    "Definition": "Verhältnis der Geschossfläche zur Grundstücksfläche",
    "Zeichen": "Zahl (z.B. 0,9)"
  },
  "Bauweise": {
    "Definition": "Art der Bebauung",
    "Zeichen": {
      "a": "offene Bauweise",
      "b": "geschlossene Bauweise"
    }
  },
  "Dachform und Dachneigung": {
    "Defini

In [8]:
# Zeichenerklärung – Inhalt zu den Hauptkategorien – NO CONTEXT
images = [ze_path, ze_two_col_path]

instruction = 'Extrahiere alle Informationen zu den folgenden Themen: Art der baulichen Nutzung, Maß der baulichen Nutzung, Bauweise, überbaubare Grundstücksfläche.'

messages = await jarvis.extractTextFromImages(instruction, images)
pprint(messages)

Hier sind die extrahierten Informationen zu den angeforderten Themen aus dem Bebauungsplan:

**Art der baulichen Nutzung (§ 9 (1) 1 BBauG):**
- **Wohnbauflächen:**
  - WR: Reines Wohngebiet (§ 3 BauNVO)
  - WA: Allgemeines Wohngebiet (§ 4 BauNVO)
- **Mischgebiet:**
  - MI: Mischgebiet (§ 6 BauNVO)
- **Gewerbegebiet:**
  - GE2: Eingeschränktes Gewerbegebiet für nicht wesentlich störende Gewerbebetriebe (§ 8 BauNVO)

**Maß der baulichen Nutzung (§ 9 (1) 1 BBauG):**
- **Zahl der Vollgeschosse (§ 18 BauNVO)**
- **Grundflächenzahl (GRZ) (§ 19 BauNVO)**
- **Geschossflächenzahl (GFZ) (§ 20 BauNVO)**

**Bauweise (§ 1 (1) 1b BBauG und § 22 BauNVO):**
- **Offene Bauweise:** Nur Einzel- und Doppelhäuser zulässig
- **Geschlossene Bauweise**

**Überbaubare Grundstücksflächen (§ 9 (1) 1b BBauG):**
- **Baugrenze (§ 23 (3) BauNVO)**

Diese Informationen geben einen Überblick über die Festsetzungen im Bebauungsplan bezüglich der Art und des Maßes der baulichen Nutzung, der Bauweise und der überbaubaren

In [9]:
# Zeichenerklärung – Inhalt zu den Hauptkategorien – WITH CONTEXT
# Kontext: Schema, BauNVO
%store -r msgbaunvo
images = [ze_path, ze_two_col_path]
schemas = parser.text2prompts([msg211_l04_zettbb_nz, msg211_l04_zebb_nz, msg211_s03_zetwocolbb_nz])
baunvo = parser.text2prompts([msgbaunvo])
contexts = list(map(lambda schema: [*baunvo,schema], schemas))

instruction = 'Extrahiere alle Informationen zu den folgenden Themen: Art der baulichen Nutzung, Maß der baulichen Nutzung, Bauweise, überbaubare Grundstücksfläche.'

message = await jarvis.extractTextFromImagesWithContexts(instruction, images, contexts)
pprint(message)

### Art der baulichen Nutzung

- **Bauland und Art der Nutzung (§ 9 (1) 1 BBauG)**
  - **Wohnbauflächen**
    - **WR**: Reines Wohngebiet (§ 3 BauNVO)
    - **WA**: Allgemeines Wohngebiet (§ 4 BauNVO)
  - **Gemischte Bauflächen**
    - **MI**: Mischgebiet (§ 6 BauNVO)
  - **Gewerbebetriebe**
    - **GE2**: Eingeschränktes Gewerbegebiet für nicht wesentlich störende Gewerbebetriebe (§ 8 BauNVO)

### Maß der baulichen Nutzung

- **Maß der baulichen Nutzung (§ 9 (1) 1 BBauG)**
  - **Zahl der Vollgeschosse (§ 18 BauNVO)**
  - **Grundflächenzahl (GRZ) (§ 19 BauNVO)**
  - **Geschossflächenzahl (GFZ) (§ 20 BauNVO)**

### Bauweise

- **Bauweise (§ 1 (1) 1b BBauG und § 22 BauNVO)**
  - **offene Bauweise**: nur Einzel- und Doppelhäuser zulässig
  - **geschlossene Bauweise**

### Überbaubare Grundstücksfläche

- **Überbaubare Grundstücksflächen (§ 9 (1) 1b BBauG)**
  - **Baugrenze (§ 23 (3) BauNVO)**

### Füllschema der Nutzungsschablone (§ 9 (1) 1a BBauG)

```
+-------------------+--------------